In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# ======================== CELL 1: DEPENDENCIES ==========================
import subprocess
import sys

print('🔧 Installing dependencies...')
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "pyarrow"], capture_output=True, check=False)
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "pyarrow==15.0.2", "keybert", "rank-bm25", "faiss-cpu", "sacremoses"], check=True)
print('✅ Dependencies installed. RESTART kernel and run Cell 2.')


In [1]:
# ===================== CELL 2: IMPORTS & SETUP =====================
import warnings
warnings.filterwarnings("ignore")

import re
import json
import time
import pickle
from pathlib import Path
from dataclasses import dataclass
from typing import List, Dict, Tuple

import numpy as np
import torch
import faiss
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from nltk.tokenize import word_tokenize, sent_tokenize
from rank_bm25 import BM25Okapi

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔧 Using device: {device}")

# ============================================================================
# CONFIGURATION FOR PEDIATRICS DOMAIN
# ============================================================================

@dataclass
class DomainConfig:
    name: str = "pediatrics"
    dataset_name: str = "medalpaca/medical_meadow_wikidoc"  # ✅ CORRECTED
    dataset_split: str = "train"
    chunk_window: int = 3
    chunk_stride: int = 1
    embed_model: str = "sentence-transformers/all-MiniLM-L6-v2"

# Pediatric filtering keywords
PEDIATRIC_KEYWORDS = [
    'pediatric', 'child', 'children', 'infant', 'baby', 'newborn',
    'toddler', 'adolescent', 'kid', 'vaccination', 'vaccine',
    'growth', 'development', 'year old', 'years old'
]

config = DomainConfig()

print(f"📋 Building index for: {config.name}")
print(f"📦 Dataset: {config.dataset_name}")
print(f"🔍 Will filter for pediatric keywords")


2025-11-03 13:21:28.035773: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762176088.059015     140 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762176088.066241     140 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🔧 Using device: cuda
📋 Building index for: pediatrics
📦 Dataset: medalpaca/medical_meadow_wikidoc
🔍 Will filter for pediatric keywords


In [2]:
# ===================== CELL 3: DATASET LOADING WITH PEDIATRIC FILTERING =====================

def is_pediatric_content(text: str) -> bool:
    """Check if text contains pediatric keywords."""
    text_lower = text.lower()
    return any(keyword in text_lower for keyword in PEDIATRIC_KEYWORDS)

def extract_qa_pairs(dataset) -> list:
    """Extract Q&A pairs from WikiDoc and filter for pediatrics."""
    qa_data = []
    filtered_count = 0
    
    for idx, row in enumerate(dataset):
        try:
            question = ""
            answer = ""
            
            # MedAlpaca WikiDoc structure
            if 'instruction' in row and 'output' in row:
                question = str(row['instruction']).strip()
                answer = str(row['output']).strip()
            
            elif 'input' in row and 'output' in row:
                question = str(row['input']).strip()
                answer = str(row['output']).strip()
            
            elif 'question' in row and 'answer' in row:
                question = str(row['question']).strip()
                answer = str(row['answer']).strip()
            
            # Validate
            if not (question and answer and len(answer) > 20):
                continue
            
            # Filter for pediatric content
            combined_text = question + " " + answer
            if is_pediatric_content(combined_text):
                qa_data.append({
                    "question": question,
                    "answer": answer,
                    "source_id": idx
                })
            else:
                filtered_count += 1
                
        except Exception as e:
            if idx < 3:
                print(f"⚠️ Row {idx} skipped: {e}")
            continue
    
    return qa_data, filtered_count

# Load dataset
print(f"📥 Loading {config.dataset_name}...")
dataset = load_dataset(config.dataset_name, split=config.dataset_split)
print(f"✅ Loaded {len(dataset)} rows")

# Print sample row to verify structure
print(f"\n📋 Sample row structure:")
print(f"Keys: {list(dataset[0].keys())}")

# Extract Q&A pairs with pediatric filtering
print(f"\n🔍 Extracting pediatric Q&A pairs...")
qa_data, filtered_out = extract_qa_pairs(dataset)
print(f"✅ Extracted {len(qa_data)} pediatric Q&A pairs")
print(f"🗑️ Filtered out {filtered_out} non-pediatric entries")
print(f"📊 Retention rate: {len(qa_data)/len(dataset)*100:.1f}%")

if len(qa_data) < 100:
    print(f"⚠️ WARNING: Only {len(qa_data)} pediatric pairs extracted!")
else:
    print(f"\n📝 Sample pediatric Q&A:")
    if qa_data:
        print(f"Q: {qa_data[0]['question'][:150]}...")
        print(f"A: {qa_data[0]['answer'][:150]}...")


📥 Loading medalpaca/medical_meadow_wikidoc...


Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

✅ Loaded 10000 rows

📋 Sample row structure:
Keys: ['input', 'output', 'instruction']

🔍 Extracting pediatric Q&A pairs...
✅ Extracted 2236 pediatric Q&A pairs
🗑️ Filtered out 7739 non-pediatric entries
📊 Retention rate: 22.4%

📝 Sample pediatric Q&A:
Q: Answer this question truthfully...
A: The health care provider will perform a physical exam.
There may be signs of:
Delayed puberty Cardiomegaly Stunted growth or failure to grow Cardiomyo...


In [3]:
# ===================== CELL 4: TEXT CHUNKING =====================

def create_chunks(data: List[Dict], window: int = 3, stride: int = 1) -> List[Dict]:
    """Create sentence-level chunks from answers."""
    chunks = []
    
    for item in data:
        text = item.get("answer", "")
        if not text or len(text) < 50:
            continue
        
        sentences = sent_tokenize(text)
        if not sentences:
            continue
        
        # If answer is short, keep as one chunk
        if len(sentences) <= window:
            chunks.append({
                "chunk": " ".join(sentences),
                "source_idx": item.get("source_id", -1),
                "chunk_id": len(chunks)
            })
            continue
        
        # Create overlapping chunks
        for i in range(0, max(1, len(sentences) - window + 1), stride):
            chunks.append({
                "chunk": " ".join(sentences[i:i + window]),
                "source_idx": item.get("source_id", -1),
                "chunk_id": len(chunks)
            })
    
    return chunks

# Create chunks
print(f"🔪 Creating chunks (window={config.chunk_window}, stride={config.chunk_stride})...")
chunks = create_chunks(qa_data, window=config.chunk_window, stride=config.chunk_stride)
print(f"✅ Created {len(chunks)} chunks")

# Sample chunks
print("\n📝 Sample chunks:")
for i, chunk in enumerate(chunks[:3]):
    print(f"{i+1}. {chunk['chunk'][:150]}...")


🔪 Creating chunks (window=3, stride=1)...
✅ Created 19888 chunks

📝 Sample chunks:
1. The health care provider will perform a physical exam. There may be signs of:
Delayed puberty Cardiomegaly Stunted growth or failure to grow Cardiomyo...
2. Secondary prevention occurs once the disease has occurred and aims to prevent progression and complication development. Protective measures include co...
3. Protective measures include controlling the tidal volume, plateau pressure, PEEP, and FiO2. Measures taken to reduce the risk of aspiration such as el...


In [4]:
# ===================== CELL 5: BUILD FAISS INDEX =====================

# Load embedder
print(f"📦 Loading embedder: {config.embed_model}")
embedder = SentenceTransformer(config.embed_model, device=device)
print(f"✅ Embedder loaded")

# Extract chunk texts
id2doc = [chunk["chunk"] for chunk in chunks]
print(f"📊 Encoding {len(id2doc)} chunks...")

# Create embeddings
embeddings = embedder.encode(
    id2doc,
    normalize_embeddings=True,
    show_progress_bar=True,
    batch_size=32,
    convert_to_numpy=True
).astype('float32')

print(f"✅ Embeddings shape: {embeddings.shape}")

# Build FAISS index
dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embeddings)

print(f"✅ FAISS index built: {index.ntotal} vectors, dimension {dim}")


📦 Loading embedder: sentence-transformers/all-MiniLM-L6-v2


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Embedder loaded
📊 Encoding 19888 chunks...


Batches:   0%|          | 0/622 [00:00<?, ?it/s]

✅ Embeddings shape: (19888, 384)
✅ FAISS index built: 19888 vectors, dimension 384


In [5]:
# ===================== CELL 6: BUILD BM25 INDEX =====================

# Tokenize for BM25
print(f"🔨 Building BM25 index...")
bm25_corpus = [word_tokenize(doc.lower()) for doc in id2doc]
bm25 = BM25Okapi(bm25_corpus)
print(f"✅ BM25 index built")


🔨 Building BM25 index...
✅ BM25 index built


In [6]:
# ===================== CELL 7: SAVE INDEXES =====================

# File names
index_file = f"{config.name}_faiss.index"
id2doc_file = f"{config.name}_id2doc.pkl"
metadata_file = f"{config.name}_metadata.json"

# Save FAISS index
print(f"💾 Saving FAISS index to {index_file}...")
faiss.write_index(index, index_file)

# Save id2doc mapping
print(f"💾 Saving id2doc to {id2doc_file}...")
with open(id2doc_file, "wb") as f:
    pickle.dump(id2doc, f)

# Save metadata
metadata = {
    "created_at": time.time(),
    "domain": config.name,
    "dataset": config.dataset_name,
    "n_vectors": int(index.ntotal),
    "embedding_dim": dim,
    "chunk_window": config.chunk_window,
    "chunk_stride": config.chunk_stride,
    "embed_model": config.embed_model
}

print(f"💾 Saving metadata to {metadata_file}...")
with open(metadata_file, "w") as f:
    json.dump(metadata, f, indent=2)

print("\n" + "="*80)
print("✅ INDEX BUILDING COMPLETE!")
print("="*80)
print(f"📁 Files created:")
print(f"  - {index_file}")
print(f"  - {id2doc_file}")
print(f"  - {metadata_file}")
print(f"\n📊 Statistics:")
print(f"  - Total vectors: {index.ntotal}")
print(f"  - Embedding dimension: {dim}")
print(f"  - Source Q&A pairs: {len(qa_data)}")
print(f"  - Chunks created: {len(chunks)}")
print("="*80)


💾 Saving FAISS index to pediatrics_faiss.index...
💾 Saving id2doc to pediatrics_id2doc.pkl...
💾 Saving metadata to pediatrics_metadata.json...

✅ INDEX BUILDING COMPLETE!
📁 Files created:
  - pediatrics_faiss.index
  - pediatrics_id2doc.pkl
  - pediatrics_metadata.json

📊 Statistics:
  - Total vectors: 19888
  - Embedding dimension: 384
  - Source Q&A pairs: 2236
  - Chunks created: 19888


In [7]:
# ===================== CELL 8: VERIFY INDEX (OPTIONAL) =====================

def test_retrieval(query: str, top_k: int = 5):
    """Test the built index with a sample query."""
    print(f"\n🔍 Testing query: '{query}'")
    
    # Embed query
    query_emb = embedder.encode([query], normalize_embeddings=True, convert_to_numpy=True).astype('float32')
    
    # Search FAISS
    D, I = index.search(query_emb, top_k)
    
    print(f"\n📋 Top {top_k} results:")
    for i, (idx, score) in enumerate(zip(I[0], D[0]), 1):
        print(f"\n{i}. Score: {score:.4f}")
        print(f"   {id2doc[idx][:200]}...")

# Test queries for pediatrics
test_queries = [
    "What are the vaccination schedules for infants?",
    "My 3 year old has a fever, what should I do?",
    "Normal developmental milestones for toddlers"
]

print("\n" + "="*80)
print("🧪 TESTING PEDIATRICS INDEX")
print("="*80)

for query in test_queries:
    test_retrieval(query, top_k=3)



🧪 TESTING PEDIATRICS INDEX

🔍 Testing query: 'What are the vaccination schedules for infants?'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


📋 Top 3 results:

1. Score: 0.6285
   Vaccination should be offered as long as influenza viruses are circulating. Children aged 6 months through 8 years who require 2 doses should receive their first dose as soon as possible after vaccine...

2. Score: 0.6207
   Children aged 6 months through 8 years who require 2 doses should receive their first dose as soon as possible after vaccine becomes available, and the second dose ≥4 weeks later. Algorithm adapted fr...

3. Score: 0.6172
   Health care providers should offer vaccination soon after vaccine becomes available (by October, if possible). Vaccination should be offered as long as influenza viruses are circulating. Children aged...

🔍 Testing query: 'My 3 year old has a fever, what should I do?'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


📋 Top 3 results:

1. Score: 0.6280
   He was in a good state of health until about 3 days ago when he developed a very high fever. The mother says the temperature was as high as 40C (104F) when she measured it with her thermometer at home...

2. Score: 0.5821
   She gave him some tylenol and the fever subsided after which the rash developed. It started as a non itchy pink rash with rose spots on the head and is now generalized all over the body. Today the boy...

3. Score: 0.5519
   You can take these steps at home:
Control your fever with aspirin, nonsteroidal anti-inflammatory drugs (NSAIDs, such as ibuprofen or naproxen), or acetaminophen. DO NOT give aspirin to children. Do n...

🔍 Testing query: 'Normal developmental milestones for toddlers'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


📋 Top 3 results:

1. Score: 0.5597
   Assurances that this condition will clear as the baby matures are very common. However, studies have shown that the condition not infrequently persists into the toddler years, and less commonly into l...

2. Score: 0.5133
   Development is typically normal until 6-18 months, when language and motor milestones regress, purposeful hand use is lost and acquired deceleration in the rate of head growth (resulting in microcepha...

3. Score: 0.5095
   Refer to pediatric specialist and development intervention programs if indicated. Check for developmental delays and learning difficulties, especially visual spatial deficits. A thorough evaluation pr...
